In [ ]:
import transformers
import os
import pandas as pd
import datasets
os.environ["CUDA_VISIBLE_DEVICES"]="5"
print(transformers.__version__)
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm_notebook as tqdm
import time
import datetime
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np
from nltk.translate.bleu_score import corpus_bleu
import numpy as np
rougef = datasets.load_metric('rouge')
bleu = datasets.load_metric('bleu')
meteor = datasets.load_metric('meteor')
print(device)



In [2]:
model_add = ['/local/data1/users/omidvar/HG/BART_large_cnn/results/models/bart-large-cnn-NewsRoomSmall/checkpoint-194804','BART_largeCNN_finetuned_NS']#bart finetuned
# model_add = ['facebook/bart-large-cnn','BART_largeCNN_pretrained_NS']
# model_add = ['/local/data1/users/omidvar/HG/prophetnet_large_cnndm/results/models/prophetnet-large-uncased-cnndm-NewsRoom/checkpoint-97402','Prophetnet_large_cnndm_finetuned_NS']
# model_add = ['microsoft/prophetnet-large-uncased-cnndm','Prophetnet_large_cnndm_pretrained_NS']
# model_add = ['/local/data1/users/omidvar/HG/T5Supervised1/results/models/t5-small-NewsRoom1/checkpoint-487005', 'T5_small_finetuned_NS']
# model_add = ['t5-small', 'T5_small_pretrained_NS']




data_files = [f'/local/data1/users/omidvar/HG/Datasets/TwitterPopular/df_twitter_testvalid.feather']

data_file_name = ['popular']
run_tokenizer = True
run_generator = True

#Generator
small_data =  ''#'' #'_small' 
filtered = '' #'_filtered' ''

### Tokenizer


In [3]:
def Tokenizer(run_tokenizer):
    if not run_tokenizer:
        return 0
    tokenizer = AutoTokenizer.from_pretrained(model_add[0])
    cnt = 0
    data = pd.read_feather(data_files[0])
    max_input_length = 1024
    for file in tqdm(data_files):
        data = pd.read_feather(file)

        print(f'len(data) = {len(data)}')
#         data = data.iloc[0:15]
        text = data['target_paragraphs'].tolist()
        text = ["headline: " + x for x in text]

        encoding = tokenizer(text
                             , max_length=max_input_length
                             , truncation=True
                             ,return_tensors="pt"
                            ,padding='longest')

        input_ids = encoding['input_ids']
        attention_masks = encoding['attention_mask']
        print(f'type(input_ids) = {type(input_ids)}')
        print(f'len(input_ids) = {len(input_ids)}')
        print(f'input_ids[0]) = {input_ids[0]}')
        print(f'input_ids.shape  = {input_ids.shape}')
        
        log_path = os.path.join('./Res',model_add[1])
        if not os.path.exists(log_path):
            os.mkdir(log_path)
        
        torch.save(input_ids
                   , os.path.join(log_path,f'input_ids_{data_file_name[cnt]}.pt'))
        torch.save(attention_masks
                   , os.path.join(log_path,f'attention_masks_{data_file_name[cnt]}.pt'))
        cnt+=1

### Generator

In [4]:
def append_result(result, all_results_list,number_of_samples):
    result_turn = 0
    for i in range(len(result)):
        all_results_list[result_turn].append(result[i])
        
        result_turn+=1
        if(result_turn == number_of_samples):
            result_turn = 0
    return all_results_list

In [5]:
def save_results(all_results_list
                 , number_of_samples
                 , file
                 , log_path
                ,num_beams
                ,small_data
                ,filtered):
    
    
    #datestring = 'test'
    
    with open(log_path+f'/log_{file}.txt', 'w') as f:
        f.write(f'number_of_samples = {number_of_samples}\n')
        
#         f.write(f'exec_time = {exec_time} \n')
        f.write(f'num_beams = {num_beams} \n')
        f.write(f'num_beams = {small_data} \n')
        
    for i in range(number_of_samples):
        results = all_results_list[i]
        
        df_results = pd.DataFrame(results,columns=['Title'])
        
        df_results.to_csv(os.path.join(log_path, f'results_{file}{small_data}{filtered}_{i}.csv')
                          ,index= False
                          ,header=False)
        df_results.reset_index().to_feather(os.path.join(log_path, f'results_{file}{small_data}{filtered}_{i}.feather'))

In [6]:
def generator(run_generator):
    if not run_generator:
        return 0
    

    number_of_samples = 10
    num_beams = 4
    
    small_data =  ''#'' #'_small' 
    filtered = '' #'_filtered' ''
    
    results = []
    
    start_time = time.time()
    datestring = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # datestring = 'test'
    log_path = os.path.join('./Res',model_add[1],datestring)
    if not os.path.exists(log_path):
        os.mkdir(log_path)
    
    data_path = os.path.join('./Res',model_add[1])
    for file in tqdm(data_file_name):
        results = []
        input_ids = torch.load(os.path.join(data_path,f'input_ids_{file}{small_data}{filtered}.pt'))
        attention_masks = torch.load(os.path.join(data_path,f'attention_masks_{file}{small_data}{filtered}.pt'))
        print(f'{file} is {input_ids.shape}')
        
    # from transformers import T5ForConditionalGeneration
    # model = T5ForConditionalGeneration.from_pretrained(model_add)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_add[0])
    tokenizer = AutoTokenizer.from_pretrained(model_add[0])
    model = model.to(device)
    
    for file in tqdm(data_file_name):
    #file size should not be divisible by batch size
        if file == 'test':
            batch_size = 6 #3
        else:
            batch_size = 6 #3

        all_results_list = []
        for i in range(number_of_samples):
            results = []
            all_results_list.append(results)

        results = []

        input_ids = torch.load(os.path.join(data_path,f'input_ids_{file}{small_data}{filtered}.pt'))
        attention_masks = torch.load(os.path.join(data_path,f'attention_masks_{file}{small_data}{filtered}.pt'))


        #print('input_ids.device=',input_ids.device)
        #torch.cuda.empty_cache()
        #input_ids = input_ids[0:505]
        #attention_masks = attention_masks[0:505]

        print('input_ids.shape = ',input_ids.shape)

    #     print(input_ids.shape)

        batch_step = 0
        for b in tqdm(range(int(input_ids.shape[0] / batch_size)+1)):
            batch_step=b * batch_size

            input_ids1 = input_ids[batch_step:batch_step+batch_size]
            #print('input_ids1.shape = ', input_ids1.shape)
            input_ids1 = input_ids1.to(device)
            #print(batch_step,batch_step+batch_size)

            attention_masks1 = attention_masks[batch_step:batch_step+batch_size]
            attention_masks1 = attention_masks1.to(device)
            #print('attention_masks1.shape = ', attention_masks1.shape)

    #         print(f'input_ids1.shape = {input_ids1.shape}')
    #         print(f'attention_masks1.shape = {attention_masks1.shape}')

            beam_outputs = model.generate(
                                    input_ids = input_ids1,
                                    attention_mask = attention_masks1
                                    ,do_sample = True
                                    ,num_beams = num_beams
                                    ,max_length = 20
                                    ,min_length = 1
                                    ,num_return_sequences = number_of_samples
                                    #num_beams = 4,
                                    #early_stopping = True,
                                    #num_beam_groups = 4
                                        )

            result = tokenizer.batch_decode(beam_outputs,
                                            skip_special_tokens=True)
            #print('len(result)=',len(result))


            append_result(result, all_results_list,number_of_samples)

            results= results+result
 
    #         break
    #     break

    #     print('len(results) = ',len(results))
        end_time = time.time()
        exec_time = end_time-start_time

        save_results(all_results_list=all_results_list
                    ,number_of_samples= number_of_samples
                    ,file = file
                    ,log_path = log_path
                    ,small_data=small_data
                    ,filtered=filtered
                    ,num_beams = num_beams)
    return datestring


### Best Picker

In [7]:

def batch_creator(idx, all_samples, number_of_samples):
    batch = []
    for i in range(number_of_samples):
        df_data = all_samples[i]
        title = df_data.iloc[idx]['Title']
        batch.append(title)
    return batch

def tokenizer_all(sentences, tokenizer):
    input_ids = []
    attention_masks = []
    for sent in sentences:
        encoded_input = tokenizer.encode_plus(
                                sent,
                                add_special_tokens= True,
                                max_length= 52,
                                pad_to_max_length=True,
                                return_attention_mask= True,
                                return_tensors = 'pt')

        input_ids.append(encoded_input['input_ids'])


        attention_masks.append(encoded_input['attention_mask'])
        
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
        
    return input_ids, attention_masks

def bestpicker(date_time, run_bestpicker):
    
    if not run_bestpicker:
        return 0
    
    data_address = os.path.join('./Res',model_add[1],date_time)
    best_titles_list = []
    best_titles_idx = []
    all_titles_list= []
    number_of_samples = 10
    filetype = 'popular' #'test' #'dev'
    filtered = '' # '' '_filtered'
    model_address = '/local/data1/users/omidvar/HG/twitter/HQG5/Twitter/models/sentence-transformers_all-mpnet-base-v2/saved/1'
    model_name = 'sentence-transformers/all-mpnet-base-v2'
    tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)
    model = AutoModelForSequenceClassification.from_pretrained(
    model_address, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 1, # The number of output labels--1 for regression.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    #cache_dir= '/local/data1/omidvar/codes/HG/transformercashe'
    )
    model.cuda(device)
        
        
        
        
    all_samples = []
    for i in range(number_of_samples):
        data_file = os.path.join(data_address
                                 , f'results_{filetype}{filtered}_{str(i)}.feather')
        df_data = pd.read_feather(data_file)
        all_samples.append(df_data)
        
    model.eval()    
    
    data_length = len(all_samples[0])
        
        
    for i in tqdm(range(data_length)):
        #read the batch
        batch  = batch_creator(i,all_samples, number_of_samples)

        #convert batch to tensors
        input_ids, attention_masks = tokenizer_all(batch
                                                  ,tokenizer)

        #get the popularity scores
        b_input_ids = input_ids.to(device)
        b_attention_masks = attention_masks.to(device)
        result = model(b_input_ids
                      ,attention_mask = b_attention_masks)

    #     if verbose:
    #         print(result)
        #get the best
        largest = np.argmax(result[0].tolist())
        best_titles_idx.append(largest)
        #best title
        best_title = batch[largest]
        #add the best title
        best_titles_list.append(best_title)



        #all titles
        for j in range(len(result[0].tolist())):
            title = batch[j]
            title_pop_score = result[0].tolist()[j]
            if j == largest:
                is_best_score = True
            else:
                is_best_score = False

            all_titles_list.append([title, title_pop_score, is_best_score])
        
    #save the best results
    df_best_titles = pd.DataFrame(best_titles_list
                                 ,columns = ['Title'])
    df_best_titles.to_csv(os.path.join(data_address,f'best_titles_{filetype}{filtered}.csv')
                         ,index=False)
    df_best_titles.to_feather(os.path.join(data_address,f'best_titles_{filetype}{filtered}.feather'))
    
        
        
    #save all titles
    df_all_titles = pd.DataFrame(all_titles_list
                                 ,columns = ['Title', 'title_pop_score', 'is_best_score'])
    df_all_titles.to_csv(os.path.join(data_address,f'all_titles_{filetype}{filtered}.csv')
                         ,index=False)
    df_all_titles.to_feather(os.path.join(data_address,f'all_titles_{filetype}{filtered}.feather'))

### Evaluator

In [10]:
def bleu_scoreh( predicts, references):
    
    
    predicts = [pre.split(' ') for pre in predicts]
    references = [[ref.split(' ')] for ref in references]
    
    score = bleu.compute(predictions=predicts,
                references= references)
#     print(score)
    return score

def bleu_function(pre_titles, ref_titles):
#     print('Calculationg Bleue scores for all is began')
    res = bleu_scoreh(pre_titles, ref_titles)
#     print('Calculationg Bleue scores for all is done!')
    res_list = []
    res_list.append(res['bleu'])
    res_list.append(res['precisions'][0])
    res_list.append(res['precisions'][1])
    res_list.append(res['precisions'][2])
    res_list.append(res['precisions'][3])
    return res_list
#     df_res  = pd.DataFrame([res_list]
#                             ,columns=['bleu'
#                                     ,'bleu_1'
#                                     ,'bleu_2'
#                                     ,'bleu_3'
#                                     ,'bleu_4'])
#     df_res.to_csv(f'./Input_files/{filename}/bleuTotalscores.csv', index= False)
  
    
#calculate meteor score for all
def meteor_function(pre_titles, ref_titles):
#     print('Meteor scores for all began')
    res = meteor.compute(predictions = pre_titles,
                         references = ref_titles)    
#     print('Meteor scores for all is done!')
    return res['meteor']

    
#calculate rouge for all 1
def rouge_function1(pre_titles, ref_titles):
    
    reward = rougef.compute(predictions = pre_titles
                            ,references = ref_titles
                           ,rouge_types = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
                           )
    rouge1_scores = [score.fmeasure for score in reward['rouge1']]
    rouge2_scores = [score.fmeasure for score in reward['rouge2']]
    rougeL_scores = [score.fmeasure for score in reward['rougeL']]
    rougeLsum_scores = [score.fmeasure for score in reward['rougeLsum']]
    
    rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
    rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
    rougeL_score = sum(rougeL_scores) / len(rougeL_scores)
    rougeLsum_score = sum(rougeLsum_scores) / len(rougeLsum_scores)
    
#     print('rouge scores1 for all is done')
    
    return [rouge1_score
            ,rouge2_score
           ,rougeL_score
           ,rougeLsum_score]
#     with open(f'./Input_files/{filename}/rougeTotalscores1.txt','w') as f:
#         print(f'rouge1 = {rouge1_score}, rouge2 = {rouge2_score}, rougeL = {rougeL_score}, rougeLsum = {rougeLsum_score}', file = f)



In [12]:
def evaluator(run_evaluator
             , date_time
             ,score_type = 'popular' #unpopular popular
             ,filetype = 'popular'
             ,fav_count_limit = 0.8 #<1.5    >0.8
             ,max_sim_limit = 0.6
             ):
    
    if not run_evaluator:
        return 0
    
    ref_add = data_files[0]
    ref_titles = pd.read_feather(ref_add)
    
    if score_type == 'popular':
        data_filter = (ref_titles['q9_favorite_count_norm'] > fav_count_limit) & (ref_titles['max_sim'] < max_sim_limit)
    elif score_type == 'unpopular':
        data_filter = (ref_titles['q9_favorite_count_norm'] < fav_count_limit) & (ref_titles['max_sim'] < max_sim_limit)
        
        
    ref_titles = ref_titles[data_filter]
    ref_titles = ref_titles['post_text'].tolist()
#     ref_titles = [ref_titles[0]]
    
    folder = os.path.join('./Res',model_add[1],date_time)
    
    all_scores = []
    
    with open(os.path.join(folder,f'all_rouge_scores_{score_type}.txt'),'w') as f:
        set_id = 0
        scores = []
        scores.append(str(set_id))
        
        pre_titles_add = os.path.join(folder,f'results_{filetype}_{set_id}.feather')
        pre_titles = pd.read_feather(pre_titles_add)
        pre_titles = pre_titles[data_filter]
        pre_titles = pre_titles['Title'].tolist()
#             print('len(ref_titles) = ',len(ref_titles))
#             print('len(pre_titles) = ',len(pre_titles))
        
        #Rouge scores
        rouge_score = rouge_function1(pre_titles, ref_titles)
        scores = scores + rouge_score
        
        #Bleu scores
        bleu_score = bleu_function(pre_titles= pre_titles
                                    , ref_titles= ref_titles)
        scores = scores + bleu_score
        
        #Meteor
        meteor_score = meteor_function(pre_titles= pre_titles
                                        , ref_titles= ref_titles)
        scores.append(meteor_score)
#             print(len(scores))
        all_scores.append(scores)
        
        f.write(f'set_id = {set_id}, rouge1_score = {rouge_score[0]} \n')
        
        #getting the average of all sets
        avg_of_sets = np.mean([x[1:] for x in all_scores]
                              , axis=0).tolist() 
        avg_of_sets = ['avg_sets'] + avg_of_sets
        all_scores.append(avg_of_sets)
    
    
    with open(os.path.join(folder,f'all_rouge_scores_{score_type}.txt'),'a') as f: 
        scores = []
        scores.append('best titles')
        #best keeper
        best_titles_add = os.path.join(folder,f'best_titles_{filetype}.feather')
        best_titles = pd.read_feather(best_titles_add)
        best_titles = best_titles[data_filter]
        best_titles = best_titles['Title'].tolist()
        #         print(len(pre_titles))

        #Rouge scores
        rouge_score = rouge_function1(best_titles, ref_titles)
        scores = scores + rouge_score

        #Bleu scores
        bleu_score = bleu_function(pre_titles= best_titles
                                  , ref_titles= ref_titles)
        scores = scores + bleu_score

        #Meteor
        meteor_score = meteor_function(pre_titles= best_titles
                                      , ref_titles= ref_titles)
        scores.append(meteor_score)
#         print(len(scores))
        all_scores.append(scores)
        f.write(f'best_titles = , rouge_score = {rouge_score} \n')    
    
    
    with open(os.path.join(folder,f'all_rouge_scores_{score_type}.txt'),'a') as f: 
        scores = []
        scores.append('worst titles')
        #best keeper
        worst_titles_add = os.path.join(folder,f'worst_titles_{filetype}.feather')
        worst_titles = pd.read_feather(worst_titles_add)
        worst_titles = worst_titles[data_filter]
        worst_titles = worst_titles['Title'].tolist()
        #         print(len(pre_titles))

        #Rouge scores
        rouge_score = rouge_function1(worst_titles, ref_titles)
        scores = scores + rouge_score

        #Bleu scores
        bleu_score = bleu_function(pre_titles= worst_titles
                                  , ref_titles= ref_titles)
        scores = scores + bleu_score

        #Meteor
        meteor_score = meteor_function(pre_titles= worst_titles
                                      , ref_titles= ref_titles)
        scores.append(meteor_score)
#         print(len(scores))
        all_scores.append(scores)
        f.write(f'worst_titles = , rouge_score = {rouge_score} \n') 
    
    df_all_scores = pd.DataFrame(all_scores,
                                columns=['setid'
                                         ,'rouge1_score'
                                        ,'rouge2_score'
                                        ,'rougeL'
                                        ,'rougeLSum'
                                        ,'bleu'
                                        ,'bleu_1'
                                        ,'bleu_2'
                                        ,'bleu_3'
                                        ,'bleu_4'
                                        ,'meteor'])
    
    
    all_scores.sort(key=lambda x: x[1], reverse= False)
    df_all_scores_add = os.path.join(folder,f'df_all_scores_{score_type}.csv')
    df_all_scores.to_csv(df_all_scores_add, index = False)
        
    return all_scores, df_all_scores

In [ ]:
Tokenizer(True)

In [ ]:
date_time = generator(True)

In [ ]:
bestpicker(date_time, True)

In [18]:

all_rouge_scores, df_all_scores = evaluator(run_evaluator = True
                                             , date_time = date_time
                                             ,score_type = 'popular' #unpopular popular
                                             ,filetype = 'popular'
                                             ,fav_count_limit = 0.8 #<1.5    >0.8
                                             ,max_sim_limit = 0.6
                                             ,run_bleu = True
                                             ,run_rouge = True
                                             ,run_meteor= True)

all_rouge_scores, df_all_scores = evaluator(run_evaluator = True
                                             , date_time = date_time
                                             ,score_type = 'unpopular' #unpopular popular
                                             ,filetype = 'popular'
                                             ,fav_count_limit = 1.5 #<1.5    >0.8
                                             ,max_sim_limit = 0.6
                                             ,run_bleu = True
                                             ,run_rouge = True
                                             ,run_meteor= True)

CPU times: user 1min 13s, sys: 692 ms, total: 1min 13s
Wall time: 1min 13s
